# CW2: Thailand's Exports to the US, Sakuljai Suwankeeree


## Table of Links
### Table
| Description | Link |
| -- | -- |
| Reflective blog | https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=46479 |
|                 | https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=46493 |
|                 | https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=46587 |
| Chosen Dataset | https://catalog.data.gov/dataset/u-s-food-imports |
| GitHub Link | https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/tree/main |

## Table of Contents
1. Executive Summary
2. Background to the Project
3. Project Dashboard 
4. Articulation of Decision Making Process
5. Review of Analytics Methods Chosen
6. Review of Available Tools
7. Review of Chosen Datasets 
8. Visualisation of Data with Accompanying Code
9. Reflective Evaluation
10. Conclusion


## 1. Executive Summary

This work is on creating an interactive dashboard for facilitating Thailand’s food export plans to the US. Drawing on data, the dashboard gives you a clear overview of export performance, competitor analysis and growth prospects. The dashboard helps stakeholders to look for top performing products, evaluate Thailand’s position compared to the rest of the market, and find new product categories through value, price, and inflation, using U.S. food import data. 

The dashboard is equipped with a lot of analytics and visualization methods such as line charts to analyze trends, stacked bar charts to compare market share, and bubble charts to visualize multi-dimensional information. Developed with Python Dash, dashboard is interactive and user-driven with filtering by category/time, hover data and updates. It gives stakeholders, policymakers and exporters practical insights so that they can make informed decisions to grow Thailand’s exports and stay ahead in the US. This project represents the way that difficult data can be translated into a simple to use tool for data-based decision-making in sustaining export development.



## 2. Background to the Project

Exporting plays a big  role in driving Thailand's economy. As of 2022, Thailand's goods exports accounted for 61% of its GDP, with key exports including cars and parts, computers, jewellery, rubber products, and plastic pellets (GOV.UK, 2023). Beyond these sectors, Thailand is a major global food exporter, with significant contributions from products such as rice, canned tuna, sugar, meat, cassava products, and canned pineapple. Only in 2022, the value of Thailand's food exports reached an impressive US$38.8 billion (Food Export USA, n.d.).

This visualization project aims to create a dashboard to enhance Thailand’s food exports to the US. It focuses on three key strategies:
Maximize Current Strengths: Leverage high-performing product categories to scale exports.
Stay Competitive: Benchmark against competing nations to maintain market share.
Explore Potential Product Categories: Identify and target emerging opportunities in the US market.

The dashboard is designed for policymakers, exporters, and stakeholders in Thailand’s food export industry. It provides a comprehensive summary of trends over the past decade, illustrating market share dynamics and highlighting growth in key product groups. The insights aim to inform strategic decisions and optimize export performance.

## 3. Project Dashboard

1.Line Chart: Displays Thailand’s export trends across various product categories from 1999 to 2023, providing a comprehensive view of long-term performance and seasonal patterns.


2.Stacked Bar Chart: Compares the market share of Thailand’s product categories in the US with those of competitor countries, offering insights into Thailand’s position within each category.


3.Bubble Chart: Highlights emerging opportunities in the US market by analyzing products that Thailand already exports, focusing on potential growth areas based on value, price, and inflation trends.

Appendix A - Description of product

In [69]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# ----- Dashboard 1 Data -----
# Raw URL
url = "https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/raw/refs/heads/main/Thai_export_by_product.xlsx"

# Fetch the file content
response = requests.get(url)
if response.status_code == 200:
    sheet_data = pd.read_excel(BytesIO(response.content), sheet_name='Sheet3')
else:
    raise Exception(f"Failed to fetch the file. Status code: {response.status_code}")

years_columns = sheet_data.columns[2:]
export_data = sheet_data.iloc[:, 2:].copy()
export_data.insert(0, 'Product', sheet_data.iloc[:, 0])
export_data_melted = export_data.melt(id_vars='Product', var_name='Year', value_name='Export Value')
export_data_melted['Year'] = pd.to_numeric(export_data_melted['Year'], errors='coerce')
export_data_melted['Export Value'] = pd.to_numeric(export_data_melted['Export Value'], errors='coerce')

custom_colors_1 = {
    'Bird eggs 3': 'firebrick',
    'Bulk grains 2': 'orange',
    'Cashew nuts': 'gold',
    'Cereal and bakery foods 4/5': 'pink',
    'Fruit juices 4': 'blue',
    'Milled grain products 3': 'green',
    'Prepared fish and shellfish 4': 'purple',
    'Prepared tree nuts 3': 'cyan',
    'Rice and products': 'yellow',
    'Total fruit and preparations 1': 'brown',
    'Total grains and products 1': 'teal',
    'Total nuts 1': 'lightblue',
    'Tree nuts 2': 'darkblue'
}

# ----- Dashboard 2 Data -----
# Raw URL
url = "https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/raw/refs/heads/main/Competitor.xlsx"

# Fetch the file content
response = requests.get(url)
if response.status_code == 200:
    data = pd.ExcelFile(BytesIO(response.content))
    df = data.parse('Sheet1')
else:
    raise Exception(f"Failed to fetch the file. Status code: {response.status_code}")
df_market_share = data.parse('Sheet1')
df_market_share.columns = df_market_share.iloc[0]
df_market_share = df_market_share[1:]
df_market_share.columns = ['Product Category', 'Country', 'Market Share']
df_market_share['Market Share'] = pd.to_numeric(df_market_share['Market Share'], errors='coerce')

custom_colors_2 = {
    'Brazil': 'firebrick', 'Canada': 'salmon', 'China': 'orange', 'Germany': 'gold', 'India': 'khaki',
    'Indonesia': 'darkkhaki', 'Italy': 'forestgreen', 'Mexico': 'darkolivegreen', 'Netherlands': 'yellowgreen',
    'Pakistan': 'mediumaquamarine', 'Philippines': 'mediumturquoise', 'Poland': 'steelblue',
    'Thailand': 'lightskyblue', 'Turkey': 'slateblue', 'United Kingdom': 'thistle', 'Vietnam': 'palevioletred',
    'Other': '#ffffff'
}
category_order = sorted(df_market_share['Country'].unique(), key=lambda x: (x == 'Other', x))

# ----- Dashboard 3 Data -----
# Raw URL
url = "https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/raw/refs/heads/main/Value_Infation_USimports.xlsx"

# Fetch the file content
response = requests.get(url)
if response.status_code == 200:
    data = pd.ExcelFile(BytesIO(response.content))
    df = data.parse('Sheet1')
else:
    raise Exception(f"Failed to fetch the file. Status code: {response.status_code}")

df_bubble = pd.read_excel(Sheet_data3, sheet_name=sheet_name3)
column_mapping = {
    "Import Food group": "Category",
    "Value (Million dollars)": "Average Food Value (Million USD)",
    "Price Average (Dollars per metric ton)": "Average Price (Dollars/Metric Ton)",
    "Average inflation": "Average Inflation (%)"
}
df_bubble = df_bubble.rename(columns=column_mapping)

# ----- App Initialization -----
app = dash.Dash(__name__)

# ----- App Layout -----
app.layout = html.Div([
    html.H1("Dashboard for Thailand's Export Analysis", style={'text-align': 'center'}),
    
    dcc.Tabs([
        # Tab 1: Line Chart
        dcc.Tab(label="Exports Over Time", children=[
            html.Div([
                html.H3("Thailand's Exports to the US", style={'text-align': 'center'}),
                
                html.Div([
                    html.Label("Select Product Category:"),
                    dcc.Dropdown(
                        id='product-filter',
                        options=[{'label': product, 'value': product} for product in export_data_melted['Product'].unique()],
                        value=export_data_melted['Product'].unique(),
                        multi=True
                    )
                ], style={'width': '50%', 'margin': '10px auto'}),
                
                html.Div([
                    html.Label("Select Year:"),
                    dcc.RangeSlider(
                        id='year-filter',
                        min=1999, max=2023,
                        marks={year: str(year) for year in range(1999, 2024, 2)},
                        value=[1999, 2023]
                    )
                ], style={'width': '80%', 'margin': '10px auto'}),
                
                dcc.Graph(id='export-line-chart')
            ])
        ]),
        
        # Tab 2: Market Share
        dcc.Tab(label="Market Share Analysis", children=[
            html.Div([
                html.H3("Market Share Percentage by Product Category", style={'text-align': 'center'}),
                dcc.Graph(
                    figure=px.bar(
                        df_market_share,
                        x="Product Category", y="Market Share", color="Country",
                        title="Market Share Percentage by Product Category",
                        color_discrete_map=custom_colors_2,
                        category_orders={"Country": category_order},
                        barmode="stack",
                        labels={"Market Share": "Market Share (%)"},
                        template="plotly_white"
                    ).update_layout(width=1000, height=600)
                )
            ])
        ]),
        
        # Tab 3: Bubble Chart
        dcc.Tab(label="Potential Products Analysis", children=[
            html.Div([
                html.H3("US Imports: Value, Price, and Inflation", style={'text-align': 'center'}),
                dcc.Graph(
                    figure=px.scatter(
                        df_bubble,
                        x="Average Food Value (Million USD)",
                        y="Average Price (Dollars/Metric Ton)",
                        size="Average Inflation (%)",
                        color="Category",
                        hover_name="Category",
                        title="Bubble Chart: Potential Products for Export",
                        color_discrete_sequence=px.colors.qualitative.Vivid,
                        template="plotly_white"
                    ).update_layout(width=1000, height=500)
                )
            ])
        ])
    ])
])

# ----- Callbacks -----
@app.callback(
    Output('export-line-chart', 'figure'),
    [Input('product-filter', 'value'), Input('year-filter', 'value')]
)
def update_chart(selected_products, selected_year_range):
    filtered_data = export_data_melted[
        (export_data_melted['Product'].isin(selected_products)) &
        (export_data_melted['Year'] >= selected_year_range[0]) &
        (export_data_melted['Year'] <= selected_year_range[1])
    ]
    fig = px.line(
        filtered_data, x='Year', y='Export Value', color='Product',
        labels={'Export Value': 'Export Value (in million USD)', 'Year': 'Year'},
        color_discrete_map=custom_colors_1, markers=True,
        template="plotly_white"
    )
    return fig

# ----- Run the App -----
if __name__ == '__main__':
    app.run_server(debug=True, port=8090)


## 4. Articulation of Decision Making Process

**4.1 Data Selection and Evaluation**
The data that has been used to create the dashboard is U.S. food import, containing import values for edible products entering U.S. ports. This data was obtained from a reliable source, the Data.gov platform. It is published by the Economic Research Service (ERS), a reputable agency known for its thorough and consent-driven data collection processes. As this dataset is explicitly intended for public access and use. For data reliability, this dataset compiled by the U.S. Department of Commerce and Census Bureau and published by ERS ensures it has been collected from reliable sources with the necessary permissions (DATA.GOV, 2020). The structured and well-documented nature of this data supports its credibility and suitability for academic analysis.. Moreover, the dataset contains detailed food import data for Thailand and competitor countries, it facilitates a comprehensive analysis of Thailand’s export performance over time, benchmarking against competitors to evaluate market share, and identifying growth opportunities. Furthermore, it supports the exploration of potential product categories based on import trends and values. The dataset establishes a robust foundation for actionable visualizations.

**4.2 Data Cleaning and Preprocessing**
After obtaining the dataset, it has been identified with the most relevant information that can answer the objectives. This involved cleaning the dataset by removing unnecessary data and mapping essential information to streamline the visualization process. This analysis focuses on maximizing current strengths by identifying trends in products exported from Thailand to highlight strong performers and those with declining demand. To stay competitive, it maps products and competitors from other countries in the same market, analyzing market share positioning and benchmarking performance. Additionally, it explores potential products by examining average prices, product group values, and inflation rates to identify items with lower inflation, higher prices, and greater demand in the US. These preprocessing steps ensured that the data was organized, accurate, and aligned with the objectives, facilitating the creation of effective visualizations for the dashboard.

**4.3 Analysis and Visualization Design**
Decision-making required identifying the most suitable visualization methods to address the project’s objectives. A line chart is frequently used to highlight overall trends and patterns, as well as to facilitate comparisons (Few, 2004). A  stacked bar chart is for the same length comparison, which in this dashboard contains percentage information (Government Analysis Function, n.d.).  and a bubble chart is highly effective for visually representing three dimensions of data and illustrating the relationships among them (Yi, n.d.), which are this dashboard showing relationships between Value, Price, and Inflation.
Iterative Development and Tool Selection
The dashboard was created using Python Dash, a tool designed for building interactive, web-based visualizations. Dash works well with libraries like Pandas and Plotly, making it simple to process data and create dynamic charts. Pandas was used to load, clean, and organize the data, while Plotly was used to design interactive charts such as bar graphs, line charts, and bubble charts, complete with hover features and tooltips for better user interaction. Dash brought everything together by combining these charts into an easy-to-use, web-based dashboard.
The interactivity of the dashboard was a key feature. For Thailand's export dashboard enabling it to filter data by product categories and time periods ensuring stakeholders could easily explore trends, compare market shares, and analyze growth opportunities. For other dashboard, they have shown the information clearly when pointing at focusing data. This was implemented to provide a user-friendly experience.

## 5. Review of Analytics Methods Chosen

When designing this dashboard, various analytics were used to ensure that data was displayed and analyzed in a way that could fit into the goals of the dashboard. These practices ensure transparent, informative and user-centric visualizations. This data was derived by a line graph of sales trends from 1999-2023 for all products and is suited for time series analysis to view the trends over time, spot patterns and detect potential anomalies or trends in product sales. This approach is particularly useful for those stakeholders who want to learn more about the long-term performance and seasonal variations (Hyndman, 2018). The tooltips and zooming capabilities of the line graph provide additional user engagement for further analysis of individual product trends. We used a stacked bar chart to display the country market share for each product category. This visualization allows users to compare the countries side by side so that they can evaluate the relative proportion of each country to a product’s total sales. It was also reinforced by visual analytics studies that demonstrate the value of comparative visualizations for decision-making and relative evaluation (Few, 2009). When combined the data, the graph not only shows the ratio of each country’s share, it also illustarte the amount of sales for each product line to better see market position and competition. It selected a bubble chart for the three values, price, and inflation. The technique allows the user to plot the correlation between these variables at the same time using bubbles position (x and y), size, and color. These visualizations can be used for exploratory analysis, notably when dealing with multiple-variable complex relationships (Munzner, 2014). The dashboard is user interface driven so that visualizations are both informative and easy to use and comprehend. As is recommended by visualization theory (Tufte, 1983; Norman, 2013), the dashboard is simple and concise with no chartjunk that might hinder the user from getting value from the dashboard.


In summary,  all analytics approaches used are in line with a careful consideration of the data features, user needs, and visualization ethos. That way, the dashboard becomes a convenient and effective platform for understanding sales patterns, market share dynamics and multi-dimensional connections that allow stakeholders to make data-driven decisions confidently.


## 6. Review of Available Tools
If I could pick my tooling freely, Power BI and Tableau would make good choices for me as both can handle big data sets and create rich visualizations. Tableau is very good at making highly customizable, multi-dimensional charts and trend maps. It integrates with AWS and Snowflake for seamless operation on big datasets and is flexible to combine different data sources into custom dashboards. Tableau’s dynamic filters and advanced interactivity allow you to see more sophisticated relationships better. Power BI is especially fast to process large data by being query-friendly and well-integrated with big data platforms. Its Power Query makes data cleaning and preparation easy, and it can be fully customized with Python and R support if you need to. It has an easy to use interface and can be used for exploration in real-time, but may not perform well with huge datasets and intricate visualizations.

Overall, Tableau is more effective when it comes to scaling and advanced visualizations, whereas Power BI is best for data processing and real-time visualization. Both of these tools are helpful to handle a lot of data and generate meaningful visualizations.

## 7. Review of Chosen Datasets 
The datasets selected for this project encompass key metrics such as FoodValue, FoodVolume, and their respective percentages, providing a robust foundation for analyzing trade trends. These metrics are essential for identifying Thailand’s strongest-performing export categories and understanding their contribution to overall trade. Category-specific datasets (e.g., Fruits, Fish, Vegetables, Meats) offer a granular view of product performance, helping pinpoint opportunities for optimization and diversification. This aligns with Davenport's (2006) argument that granular data facilitates precise decision-making by creating actionable insights in strategic initiatives.


The reliability and relevance of these well-structured datasets were critical to the project’s success, adhering to principles of data integrity that ensure valid and trustworthy insights (Wiek, 2014). Additionally, the historical depth of the data, spanning multiple years, enables longitudinal analysis to identify cyclical patterns, growth opportunities, and emerging challenges in Thailand’s food exports to the US. However, integrating diverse metrics—such as absolute values, percentages, and pricing data—across multiple product categories presented a challenge that required thoughtful design to maintain coherence and usability in the final dashboard. Despite this, the comprehensive and detailed nature of the datasets allowed for meaningful analysis and actionable insights, highlighting both current strengths and areas for strategic improvement.

## 8. Visualisation of Data with Accompanying Code
This bubble chart visualizes the relationship between the average food value (in million USD), the average price per metric ton, and inflation rates across various product categories. It enables stakeholders to identify export categories with high potential by emphasizing those with a large market value, reasonable pricing, and manageable inflation trends. The x-axis represents the monetary value (Average Food Value), the y-axis shows costs (Average Price per Metric Ton), and bubble size conveys inflation rates. Categories are color-coded for easy differentiation, aiding stakeholders in quickly identifying the most promising opportunities.

To using this dashboard, high Average Food Value (Million USD) is the key indicator of strong market demand or size, while reasonable Average Price ensures the product remains competitive for buyers. Smaller bubble sizes, indicating lower inflation, signify stability, making the product more suitable for sustainable trade.

In [64]:
# Raw URL
url = "https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/raw/refs/heads/main/Value_Infation_USimports.xlsx"

# Fetch the file content
response = requests.get(url)
if response.status_code == 200:
    data = pd.ExcelFile(BytesIO(response.content))
    df = data.parse('Sheet1')
else:
    raise Exception(f"Failed to fetch the file. Status code: {response.status_code}")


df = pd.read_excel(Sheet_data3, sheet_name=sheet_name)

column_mapping = {
    "Import Food group": "Category",
    "Value (Million dollars)": "Average Food Value (Million USD)",
    "Price Average (Dollars per metric ton)": "Average Price (Dollars/Metric Ton)",
    "Average inflation": "Average Inflation (%)"
}
df = df.rename(columns=column_mapping)

fig = px.scatter(
    df,
    x="Average Food Value (Million USD)",
    y="Average Price (Dollars/Metric Ton)",
    size="Average Inflation (%)",
    color="Category",
    hover_name="Category",
    title="Bubble Chart: Potential Products for Export",
    labels={
        "Average Food Value (Million USD)": "Food Value (Million USD)",
        "Average Price (Dollars/Metric Ton)": "Price (Dollars/Metric Ton)",
    },
    color_discrete_sequence=px.colors.qualitative.Vivid,  
)

fig.update_layout(
    xaxis_title="Average Food Value (Million USD)",
    yaxis_title="Average Price (Dollars/Metric Ton)",
    legend_title="Product Categories",
    template="plotly_white",
    width=1000,  
    height=500,  
)

app = Dash(__name__)

app.layout = html.Div([
    html.H1("US Imports: Value, Price, and Inflation Dashboard", style={'text-align': 'center'}),
    dcc.Graph(figure=fig)  
])

if __name__ == '__main__':
    app.run_server(debug=True, port=8091)  

## 9. Reflective Evaluation
If I look back on this project, it was a really interesting, learning and adapting project. Dash, Plotly, Pandas, all these required some learning as I first learned about their advanced features and then created the visualizations. The other key problem was to select the datasets relevant for my goals, this took time and effort to find the data that was appropriate for what I wanted to deliver. Getting the data ready was no less laborious, I had to scrub and sort through multiple sheets with both junk and useful data, which involved thinking things through and working hard. The dashboard design was another issue as it was my first time to put three dashboards together. This was too tricky to manage without a hitch, but by breaking them into separate dashboards it helped make it easier to use and more visual. 


Despite these challenges, the project also taught me a lot about how data can be turned into meaningful, interactive dashboards. It helped me become more technically skilled, understood the data visualization flow and was able to solve my own problems better and had a better sense of the total process from data to dashboard.

## 10. Conclusion
In conclusion, the development of this dashboard highlights the integration of strategic data analysis, effective visualization techniques, and a focus on Thailand’s export performance to the US and its position among competitors in the market. By presenting detailed insights into export value, pricing, and competition, the dashboard provides stakeholders with a clear vision of Thailand’s export landscape in the US. This specificity not only enhances the understanding of market dynamics but also transforms complex datasets into actionable strategies for optimizing Thailand’s export potential and competitiveness in the US market.



## References
DATA.GOV. (2020, February 28). U.S. Food Imports. Data.gov; Economic Research Service, Department of Agriculture. https://catalog.data.gov/dataset/u-s-food-imports


Davenport, T. H. (2006). Competing on Analytics: The New Science of Winning. Harvard Business Review Press. 


Few, S. (2004). Graph Selection Matrix Value-Encoding Objects Points Lines Bars Boxes Featured Relationships Time Series. https://www.perceptualedge.com/articles/misc/Graph_Selection_Matrix.pdf


Few, S. (2009). Now You See It: Simple Visualization Techniques for Quantitative Analysis. Analytics Press. 


Food Export USA. (n.d.). Thailand • Food Export Association of the Midwest USA and Food Export USA–Northeast. Www.foodexport.org. https://www.foodexport.org/export-insights/market-country-profiles/thailand/


GOV.UK. (2023, June 21). Overseas Business Risk - Thailand. GOV.UK. https://www.gov.uk/government/publications/overseas-business-risk-thailand/overseas-business-risk-thailand


Government Analysis Function. (n.d.). Stacked bar charts. Analysisfunction.civilservice.gov.uk. https://analysisfunction.civilservice.gov.uk/support/communicating-analysis/introduction-to-data-visualisation-e-learning/module-7-stacked-bar-charts/


Hyndman, R. J., & Athanasopoulos, G. (2018). Forecasting: Principles and Practice. OTexts. Kramer, L. (2023, September 27). How Importing and Exporting Impacts the Economy. Investopedia. https://www.investopedia.com/articles/investing/100813/interesting-facts-about-imports-and-exports.asp


Porter, M. E. (1985). Competitive Advantage: Creating and Sustaining Superior Performance. Free Press. 


Segel, E., & Heer, J. (2010). Narrative Visualization: Telling Stories with Data. IEEE Transactions on Visualization and Computer Graphics, 16(6), 1139–1148. 


The Office of the United States Trade Representative. (n.d.). Thailand | United States Trade Representative. Ustr.gov. https://ustr.gov/countries-regions/southeast-asia-pacific/thailandTufte, E. R. (2001). 


The Visual Display of Quantitative Information. Graphics Press. Wiek, A. (2014). Data Quality and Decision-Making: Principles of Integrity. Sustainability Science Journal, 9(2), 189–200. 


Yi, M. (n.d.). A Complete Guide to Bubble Charts. Atlassian. https://www.atlassian.com/data/charts/bubble-chart-complete-guide

## Appendix A

### Table
| Product | description |
| -- | -- |
| Bird eggs 3 | Includes dried eggs, egg yolks, and egg albumin. |
| Prepared fish and shelfish 4 | Includes cured and smoked fish.  Excludes fish oils and fats. |
| Fuit juices 4 | Includes tomato juice. Quantity excludes liquid volumes (e.g. kiloliters) |
| Total grains and products 1 | Sum of bulk grains, milled grain products, and cereal and bakery preparations |
| Bulk grains 2 | Excludes corn for seed use and canary seed. |
| Miled grain products 3 | Includes wheat and rice flour, and other grain products. |
| Cereal and bakery foods 4/5 | Excludes malt extracts and food preparations, which are measured in liters. |
| Total nuts 1 | Sum of tree nuts, prepared tree nuts and ground nuts. |
| Tree nuts 2 | Shelled and in-shell.  Includes cashew nuts and coconuts. Excludes prepared nuts. |
| Prepared tree nuts 3 | Includes desiccated coconut, seeds and peanuts in nut mixtures.  Excludes water chestnuts, palm and shea nuts, copra. |